In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import warnings
#warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
#import gensim
#from gensim.models.doc2vec import Doc2Vec, LabeledSentence 
import numpy as np
import copy
import time
%matplotlib inline

# 读数据 concat

In [31]:
def load_data():
    train_ori = pd.read_table("round1_ijcai_18_train_20180301.txt",delim_whitespace=True)
    test_ori = pd.read_table('round1_ijcai_18_test_a_20180301.txt',delim_whitespace=True)
    return train_ori, test_ori

In [32]:
train_ori, test_ori = load_data()
train_ori = train_ori.drop_duplicates()#训练集去重
train_ori = train_ori.reset_index(drop = True)
train_1 = train_ori.copy(deep = True)
test_1 = test_ori.copy(deep = True)

In [33]:
print(train_ori.shape)
print(train_1.shape)
print(test_ori.shape)
print(test_1.shape)

(478111, 27)
(478111, 27)
(18371, 26)
(18371, 26)


In [34]:
train_y = train_1["is_trade"]
train_1 = train_1.drop("is_trade",axis = 1)

In [35]:
data = pd.concat([train_1,test_1])
data = data.reset_index(drop = True)
print(data.shape)

(496482, 26)


# 初步可视化与预处理

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496482 entries, 0 to 496481
Data columns (total 26 columns):
instance_id                  496482 non-null int64
item_id                      496482 non-null int64
item_category_list           496482 non-null object
item_property_list           496482 non-null object
item_brand_id                496482 non-null int64
item_city_id                 496482 non-null int64
item_price_level             496482 non-null int64
item_sales_level             496482 non-null int64
item_collected_level         496482 non-null int64
item_pv_level                496482 non-null int64
user_id                      496482 non-null int64
user_gender_id               496482 non-null int64
user_age_level               496482 non-null int64
user_occupation_id           496482 non-null int64
user_star_level              496482 non-null int64
context_id                   496482 non-null int64
context_timestamp            496482 non-null int64
context_page_id     

In [37]:
for i in data.columns:
    print(i,"|",'（distinct）',len(set(list(data[i]))))

instance_id | （distinct） 496455
item_id | （distinct） 10236
item_category_list | （distinct） 14
item_property_list | （distinct） 11181
item_brand_id | （distinct） 2075
item_city_id | （distinct） 128
item_price_level | （distinct） 14
item_sales_level | （distinct） 18
item_collected_level | （distinct） 18
item_pv_level | （distinct） 22
user_id | （distinct） 207641
user_gender_id | （distinct） 4
user_age_level | （distinct） 9
user_occupation_id | （distinct） 5
user_star_level | （distinct） 12
context_id | （distinct） 496482
context_timestamp | （distinct） 297416
context_page_id | （distinct） 20
predict_category_property | （distinct） 82378
shop_id | （distinct） 4003
shop_review_num_level | （distinct） 25
shop_review_positive_rate | （distinct） 13021
shop_star_level | （distinct） 22
shop_score_service | （distinct） 18102
shop_score_delivery | （distinct） 18151
shop_score_description | （distinct） 18204


In [38]:
data = data.drop("context_id",axis = 1)

In [39]:
for i in data.columns:
    print(i,"|",(data[i]).dtypes,"|",data.loc[0,i])

instance_id | int64 | 108641074714126964
item_id | int64 | 3412720377098676069
item_category_list | object | 7908382889764677758;5799347067982556520
item_property_list | object | 2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716
item_brand_id | int64 | 1975590437749032870
item_city_id | int64 | 3948283326616421003
item_price_level | int64 | 3
item_sales_level | int64 | 3
item_collected_level | int64 | 4
item_pv_level | int64 | 14
user_id | int64 | 4505772604969228686
user_gender_id | int64 | 1
user_age_level | int64 | 1003
user_occupation_id | int64 | 2005
user_star_level | int64 | 3003
context_timestamp |

# MEANencode

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from itertools import product

class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode

        :param n_splits: the number of splits used in mean encoding

        :param target_type: str, 'regression' or 'classification'

        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()

        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()

        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)

        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new

In [41]:
hc_features = ['user_id','item_brand_id','item_property_list',
               'predict_category_property','shop_id','item_id']

In [42]:
for i in hc_features:
    data[i] = data[i].astype(str)

In [43]:
ME = MeanEncoder(hc_features)

In [44]:
trans_train = ME.fit_transform(data.loc[:len(train_1)-1,:],train_y)

i:\anaconda\lib\site-packages\ipykernel_launcher.py:53: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
i:\anaconda\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in exp


In [45]:
pred_features = ['user_id_pred_1','item_brand_id_pred_1','item_property_list_pred_1',
                 'predict_category_property_pred_1','shop_id_pred_1','item_id_pred_1']

In [46]:
train_1 = trans_train

In [47]:
train_1

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,item_brand_id_pred_0,item_brand_id_pred_1,item_property_list_pred_0,item_property_list_pred_1,predict_category_property_pred_0,predict_category_property_pred_1,shop_id_pred_0,shop_id_pred_1,item_id_pred_0,item_id_pred_1
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.000000,4.477530e-28,0.999108,8.921304e-04,0.981189,1.881105e-02,1.000000,3.141712e-07,1.000000,3.141712e-07
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.000000,4.477530e-28,0.999108,8.921304e-04,0.981189,1.881105e-02,1.000000,3.141712e-07,1.000000,3.141712e-07
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.000000,4.477530e-28,0.999108,8.921304e-04,1.000000,3.141712e-07,1.000000,3.141712e-07,1.000000,3.141712e-07
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.000000,4.477530e-28,0.999108,8.921304e-04,0.981189,1.881105e-02,1.000000,3.141712e-07,1.000000,3.141712e-07
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.000000,4.477530e-28,0.999108,8.921304e-04,0.983152,1.684836e-02,1.000000,3.141712e-07,1.000000,3.141712e-07
5,7764762765372067286,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.000000,4.477530e-28,0.999108,8.921304e-04,0.986248,1.375198e-02,1.000000,3.141712e-07,1.000000,3.141712e-07
6,6956333474094867789,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1.000000,1.780020e-58,1.000000,8.763853e-17,1.000000,7.287395e-23,0.988636,1.136364e-02,1.000000,8.763853e-17
7,8387099821892927911,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1.000000,1.780020e-58,1.000000,8.763853e-17,1.000000,7.287395e-23,0.988636,1.136364e-02,1.000000,8.763853e-17
8,4021878205550012615,5202355029344881809,7908382889764677758;5755694407684602296,2072967855524022579;5131280576272319091;263639...,5520678735822176314,548352491538518780,8,9,10,16,...,0.990385,9.615385e-03,0.981132,1.886792e-02,1.000000,1.980919e-22,0.988636,1.136364e-02,0.981132,1.886792e-02
9,6499571365974135517,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1.000000,1.780020e-58,1.000000,8.763853e-17,0.993174,6.825939e-03,0.988636,1.136364e-02,1.000000,8.763853e-17


In [48]:
test_trans = ME.transform(data.loc[len(train_1):,:])
test_trans

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,item_brand_id_pred_0,item_brand_id_pred_1,item_property_list_pred_0,item_property_list_pred_1,predict_category_property_pred_0,predict_category_property_pred_1,shop_id_pred_0,shop_id_pred_1,item_id_pred_0,item_id_pred_1
478111,2475218615076601065,2275895163219263378,7908382889764677758;8277336076276184272,2636395404473730413;7515802706813700848;302719...,7838285046767229711,7534238860363577544,7,6,8,14,...,0.978509,2.149144e-02,1.000000,2.331391e-08,0.981188,1.881153e-02,1.000000,1.150871e-09,1.000000,2.331391e-08
478112,398316874173557226,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,0.980682,1.931846e-02,0.974206,2.579420e-02,0.996237,3.762713e-03,0.980682,1.931846e-02,0.980682,1.931846e-02
478113,6586402638209028583,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,0.980682,1.931846e-02,0.974206,2.579420e-02,0.972448,2.755181e-02,0.980682,1.931846e-02,0.980682,1.931846e-02
478114,1040996105851528465,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,0.980682,1.931846e-02,0.974206,2.579420e-02,0.970791,2.920874e-02,0.980682,1.931846e-02,0.980682,1.931846e-02
478115,6316278569655873454,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,0.980682,1.931846e-02,0.974206,2.579420e-02,0.985236,1.476415e-02,0.980682,1.931846e-02,0.980682,1.931846e-02
478116,868158305045921978,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,0.980682,1.931846e-02,0.974206,2.579420e-02,0.997627,2.373311e-03,0.980682,1.931846e-02,0.980682,1.931846e-02
478117,5713520501786699854,1171140458228333440,7908382889764677758;4879721024980945592,5131280576272319091;2636395404473730413;316326...,1547983184598012595,8072963182326625214,5,6,6,15,...,0.937189,6.281105e-02,1.000000,3.900308e-13,0.997464,2.535698e-03,0.937189,6.281105e-02,1.000000,3.900308e-13
478118,932945015407923184,7020520887593189887,7908382889764677758;2011981573061447208,5131280576272319091;1013419034389836991;519513...,8324857885829459863,1019055478500227370,7,10,11,14,...,0.956851,4.314888e-02,0.999999,7.692131e-07,0.981188,1.881153e-02,0.956851,4.314888e-02,0.999999,7.692131e-07
478119,1919197847086752313,8143258757857432437,7908382889764677758;2011981573061447208,2636395404473730413;4621934203383159480;124376...,8324857885829459863,1019055478500227370,8,10,13,15,...,0.956851,4.314888e-02,0.945942,5.405844e-02,0.992190,7.809963e-03,0.956851,4.314888e-02,0.945942,5.405844e-02
478120,304887065966615346,8143258757857432437,7908382889764677758;2011981573061447208,2636395404473730413;4621934203383159480;124376...,8324857885829459863,1019055478500227370,8,10,13,15,...,0.956851,4.314888e-02,0.945942,5.405844e-02,0.961026,3.897379e-02,0.956851,4.314888e-02,0.945942,5.405844e-02


In [49]:
test_1 = test_trans

In [50]:
print(train_1.shape)
print(test_1.shape)

(478111, 37)
(18371, 37)


In [51]:
data = pd.concat([train_1,test_1])
data = data.reset_index(drop = True)
print(data.shape)

(496482, 37)


In [52]:
data = data.drop(pred_features,axis = 1)
data

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,user_id_pred_0,item_brand_id_pred_0,item_property_list_pred_0,predict_category_property_pred_0,shop_id_pred_0,item_id_pred_0
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.000000,1.000000,1.000000,0.994941,1.000000,0.999108,0.981189,1.000000,1.000000
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.000000,1.000000,1.000000,0.990594,1.000000,0.999108,0.981189,1.000000,1.000000
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.000000,1.000000,1.000000,0.981189,1.000000,0.999108,1.000000,1.000000,1.000000
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.000000,1.000000,1.000000,0.981189,1.000000,0.999108,0.981189,1.000000,1.000000
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.000000,1.000000,1.000000,0.981189,1.000000,0.999108,0.983152,1.000000,1.000000
5,7764762765372067286,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.000000,1.000000,1.000000,0.997758,1.000000,0.999108,0.986248,1.000000,1.000000
6,6956333474094867789,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,5012,0.974878,0.976863,0.969278,0.981189,1.000000,1.000000,1.000000,0.988636,1.000000
7,8387099821892927911,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,5012,0.974878,0.976863,0.969278,0.990594,1.000000,1.000000,1.000000,0.988636,1.000000
8,4021878205550012615,5202355029344881809,7908382889764677758;5755694407684602296,2072967855524022579;5131280576272319091;263639...,5520678735822176314,548352491538518780,8,9,10,16,...,5012,0.974878,0.976863,0.969278,0.994941,0.990385,0.981132,1.000000,0.988636,0.981132
9,6499571365974135517,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,5012,0.974878,0.976863,0.969278,0.994941,1.000000,1.000000,0.993174,0.988636,1.000000


In [53]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'user_id_pred_0', 'item_brand_id_pred_0',
       'item_property_list_pred_0', 'predict_category_property_pred_0',
       'shop_id_pred_0', 'item_id_pred_0'],
      dtype='object')

# ridge regression 处理两个ID feature

In [54]:
# data.head(5)

In [55]:
# train_sub1 = data[["item_brand_id","is_trade"]].loc[:len(train_1)-1,:]

In [56]:
# train_sub1


In [57]:
# #特征分组，改变type
# level_features = ['shop_star_level','context_page_id','user_star_level',"user_age_level",
#                  'item_city_id','item_brand_id']

# 处理item_category_list这个feature

In [58]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,user_id_pred_0,item_brand_id_pred_0,item_property_list_pred_0,predict_category_property_pred_0,shop_id_pred_0,item_id_pred_0
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.0,1.0,1.0,0.994941,1.0,0.999108,0.981189,1.0,1.0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.0,1.0,1.0,0.990594,1.0,0.999108,0.981189,1.0,1.0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.0,1.0,1.0,0.981189,1.0,0.999108,1.000000,1.0,1.0


In [59]:
cate_list = list(data['item_category_list'])

In [60]:
# #分号前的category1
# cate_1_list = []
# for i in range(len(cate_list)):
#     cate_1_list.append(cate_list[i].split(";")[0])
# cate_1_list[:3]

In [61]:
# set(cate_1_list)#发现所有样本都一样 因此去掉该子字段的信息

In [62]:
#分号后的category2
cate_2_list = []
for i in range(len(cate_list)):
    cate_2_list.append(cate_list[i].split(";")[1])

In [63]:
# set(cate_2_list)#有区别 因此用该子字段的信息代替原来的feature

In [65]:
data["item_category_list"] = cate_2_list

In [66]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,user_id_pred_0,item_brand_id_pred_0,item_property_list_pred_0,predict_category_property_pred_0,shop_id_pred_0,item_id_pred_0
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.0,1.0,1.0,0.994941,1.0,0.999108,0.981189,1.0,1.0
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.0,1.0,1.0,0.990594,1.0,0.999108,0.981189,1.0,1.0
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5002,1.0,1.0,1.0,0.981189,1.0,0.999108,1.000000,1.0,1.0


In [67]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

In [68]:
def convert_data(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    user_query_day = data.groupby(['user_id', 'day']).size(
    ).reset_index().rename(columns={0: 'user_query_day'})
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index().rename(
        columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, 'left',
                    on=['user_id', 'day', 'hour'])

    return data

In [69]:
data = convert_data(data)

In [70]:
data = data.drop("time",axis = 1)
data

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,user_id_pred_0,item_brand_id_pred_0,item_property_list_pred_0,predict_category_property_pred_0,shop_id_pred_0,item_id_pred_0,day,hour,user_query_day,user_query_day_hour
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.994941,1.000000,0.999108,0.981189,1.000000,1.000000,18,10,3,3
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.990594,1.000000,0.999108,0.981189,1.000000,1.000000,18,12,8,5
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.981189,1.000000,0.999108,1.000000,1.000000,1.000000,18,3,2,2
3,937088850059189027,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.981189,1.000000,0.999108,0.981189,1.000000,1.000000,18,6,1,1
4,7975697065017708072,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.981189,1.000000,0.999108,0.983152,1.000000,1.000000,18,19,2,2
5,7764762765372067286,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.997758,1.000000,0.999108,0.986248,1.000000,1.000000,18,23,4,3
6,6956333474094867789,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,0.981189,1.000000,1.000000,1.000000,0.988636,1.000000,18,22,23,4
7,8387099821892927911,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,0.990594,1.000000,1.000000,1.000000,0.988636,1.000000,18,16,2,1
8,4021878205550012615,5202355029344881809,5755694407684602296,2072967855524022579;5131280576272319091;263639...,5520678735822176314,548352491538518780,8,9,10,16,...,0.994941,0.990385,0.981132,1.000000,0.988636,0.981132,18,2,1,1
9,6499571365974135517,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,0.994941,1.000000,1.000000,0.993174,0.988636,1.000000,18,23,4,4


# item_property_list这个feature与pred_property这个集合取交集 再除以真实property_list集合的势

In [71]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'user_id_pred_0', 'item_brand_id_pred_0',
       'item_property_list_pred_0', 'predict_category_property_pred_0',
       'shop_id_pred_0', 'item_id_pred_0', 'day', 'hour', 'user_query_day',
       'user_query_day_hour'],
      dtype='object')

In [72]:
import random

In [73]:
add_noise_features = ['user_id_pred_0', 'item_brand_id_pred_0',
                      'item_property_list_pred_0', 'predict_category_property_pred_0',
                      'shop_id_pred_0', 'item_id_pred_0']

In [83]:
for i in add_noise_features:
    noise_list = list(np.random.randn(len(data)))
    v = list(map(lambda x: x[0]+x[1], zip(noise_list, list(data[i]))))
    data[i] = v

In [84]:
data

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,user_id_pred_0,item_brand_id_pred_0,item_property_list_pred_0,predict_category_property_pred_0,shop_id_pred_0,item_id_pred_0,day,hour,user_query_day,user_query_day_hour
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.843223,1.556270,2.283851,0.909007,1.106726,0.181197,18,10,3,3
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1.927073,2.683758,1.062713,0.906457,0.602234,0.736568,18,12,8,5
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.075731,2.760199,2.134019,0.839738,1.268478,-0.006948,18,3,2,2
3,937088850059189027,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.582230,0.628485,2.131482,1.053362,0.036537,2.512860,18,6,1,1
4,7975697065017708072,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,0.434147,1.353250,3.113078,2.625311,1.326088,1.262853,18,19,2,2
5,7764762765372067286,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,2.158296,0.008786,0.621505,0.422685,1.921685,1.483271,18,23,4,3
6,6956333474094867789,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,-0.787175,-1.647097,0.903466,1.095869,0.654191,1.945038,18,22,23,4
7,8387099821892927911,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,2.044220,0.886402,-0.310107,-0.639486,-0.690078,2.503017,18,16,2,1
8,4021878205550012615,5202355029344881809,5755694407684602296,2072967855524022579;5131280576272319091;263639...,5520678735822176314,548352491538518780,8,9,10,16,...,-0.112692,2.415390,0.844922,2.779811,-0.136723,2.797396,18,2,1,1
9,6499571365974135517,285660928590172217,8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1.515726,1.172471,1.477936,-0.272829,-0.123603,1.251121,18,23,4,4


In [193]:
# pred_cate_prop_list = list(data['predict_category_property'])
# #用；分解每个cate：property
# pcp_list = []
# for i in pred_cate_prop_list:
#     pcp_list.append(i.split(';'))
# #再用：将cate，property分开
# pcp_list_2 = []
# for i in pcp_list:
#     tmp = []
#     for k in i:
#         tmp.append(k.split(":"))
#     pcp_list_2.append(tmp)
# #每一行的category
# each_category_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.append(k[0])
#     each_category_list.append(tmp)
# #每一行的property
# each_property_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.extend(k[1:])
#     each_property_list.append(tmp)
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# each_property_list_prep = [list(set(x)) for x in property_clean_1]
# print(each_category_list[:3])#预测的类别
# print("======================")
# print(each_property_list_prep[:3])#预测的属性

In [194]:
# for i in each_property_list_prep:
#     if '-1' in i:
#         i.remove('-1')
# print(each_property_list_prep[:3])#预测的属性

In [195]:
# lenth_cate_pred = []
# for i in range(len(each_category_list)):
#     lenth_cate_pred.append(len(each_category_list[i]))
# lenth_prop_pred = []
# for i in range(len(each_property_list_prep)):
#     lenth_prop_pred.append(len(each_property_list_prep[i]))

In [196]:
# print(lenth_cate_pred[:2])
# print(lenth_prop_pred[:2])

In [197]:
# data['lenth_cate_pred'] = lenth_cate_pred#int
# data['lenth_prop_pred'] = lenth_prop_pred#int

In [198]:
# complexity_list = []
# for i in range(len(data)):
#     complexity_list.append(data.loc[i,'lenth_cate_pred'] * data.loc[i,'lenth_prop_pred'])
# data['query_complexity'] = complexity_list

In [199]:
# complexity_list[:2]

In [200]:
# union_property = []
# for i in range(len(data)):
#     union_property.append(len((set(data.loc[i,"item_property_list"].split(";"))) & \
#      set(each_property_list_prep[i]))/len(set(data.loc[i,"item_property_list"].split(";"))))

In [201]:
# lenth_prop = []
# for i in range(len(data)):
#     lenth_prop.append(len(set(data.loc[i,"item_property_list"].split(";"))))

In [202]:
# data["union_property"] = union_property
# data["lenth_prop"] = lenth_prop

In [203]:
# union_category = []
# for i in range(len(data)):
#     union_category.append(len((set([data.loc[i,"item_category_list"]])) & \
#      set(each_category_list[i])))

In [204]:
# data["union_category"] = union_category
# #data["lenth_cate"] = lenth_cate

In [85]:
#添加word2vec的特征
data_w2v = pd.read_csv("data_nlp.csv")

In [86]:
w2v_features = ['prop1', 'prop2', 'prop3', 'prop4', 'prop5',
       'prop6', 'prop7', 'prop8', 'prop9', 'prop10', 'pred_cate1',
       'pred_cate2', 'pred_cate3', 'pred_cate4', 'pred_cate5', 'pred_prop1',
       'pred_prop2', 'pred_prop3', 'pred_prop4', 'pred_prop5', 'pred_prop6',
       'pred_prop7', 'pred_prop8', 'pred_prop9', 'pred_prop10']

In [87]:
data_ = pd.concat([data,data_w2v[w2v_features]],axis = 1)

In [208]:
# data_ = data_.drop(["item_property_list","predict_category_property"],axis = 1)

In [88]:
data_.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [210]:
# #分号分割后的所有属性放到一起
# property_list = list(data['item_property_list'])
# property_list_all = []
# for i in range(len(property_list)):
#     for j in range(len(property_list[i].split(";"))):
#         property_list_all.append(property_list[i].split(";")[j])
# #看看有多少种不同的属性
# property_distinct = list(set(property_list_all))
# print(len(property_distinct))

In [211]:
# #将原始的数据转换成doc2vec可用的lists of list的格式
# property_list_pre = []
# for i in range(len(property_list)):
#     temp = []
#     temp.append(property_list[i])
#     property_list_pre.append(temp)
# pre_process = [[word for word in document[0].split(';')] for document in property_list_pre]

In [212]:
# #看看拆分结果
# print(property_list_pre[:2])
# print("-"*50)
# print(pre_process[:2])

In [50]:
# def doc2vec(doc, size, window = 1, min_count = 1):
#     """
#     doc:lists of list格式
#     size:目标vector大小 
#     """
#     TaggededDocument = gensim.models.doc2vec.TaggedDocument  
#     labeled_doc = []  
#     for i, text in enumerate(doc):  
#         document = TaggededDocument(text, tags=[i])  
#         labeled_doc.append(document)
#     model_dm = Doc2Vec(labeled_doc, min_count = min_count, window = window, size = size, 
#                        negative=5, workers=4)
#     array_property = model_dm.infer_vector(pre_process[0])
#     for i in doc[1:]:
#         temp = model_dm.infer_vector(i)
#         array_property = np.vstack((array_property,temp))
#     dataframe_property = pd.DataFrame(array_property)
#     return dataframe_property

In [51]:
# dataframe_property = doc2vec(pre_process, 10)
# dataframe_property.head(3)

In [52]:
# dataframe_property.shape

In [53]:
# data.shape

In [54]:
# dataframe_property.columns = ["prop1","prop2","prop3","prop4","prop5",
#                              "prop6","prop7","prop8","prop9","prop10"]

In [55]:
# data_ = pd.concat([data, dataframe_property],axis=1)

In [56]:
# len(set(list(data["item_brand_id"])))#商品品牌暂时不处理

In [89]:
data_1 = data_.drop(['instance_id'],axis = 1)
data_1.head(3)

,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [90]:
data_1.shape

(496482, 59)

# 处理predict_category_property这个feature

In [59]:
#pred_cate_prop_list = list(data_1['predict_category_property'])

In [60]:
#pred_cate_prop_list[:3]

In [61]:
# #用；分解每个cate：property
# pcp_list = []
# for i in pred_cate_prop_list:
#     pcp_list.append(i.split(';'))
# #再用：将cate，property分开
# pcp_list_2 = []
# for i in pcp_list:
#     tmp = []
#     for k in i:
#         tmp.append(k.split(":"))
#     pcp_list_2.append(tmp)

In [62]:
# pcp_list_2[:2]

In [63]:
# #每一行的category
# each_category_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.append(k[0])
#     each_category_list.append(tmp)

In [64]:
# each_category_list[:3]

In [65]:
# dataframe_pred_cate = doc2vec(each_category_list, 5)
# dataframe_pred_cate.head(3)

In [66]:
# dataframe_pred_cate.columns = ["pred_cate1","pred_cate2","pred_cate3",
#                                "pred_cate4","pred_cate5"]

In [67]:
# data_1_ = pd.concat([data_1, dataframe_pred_cate],axis=1)
# data_1_.head(3)

In [68]:
# data_1_.shape

In [69]:
# #每一行的property
# each_property_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.extend(k[1:])
#     each_property_list.append(tmp)

In [70]:
# each_property_list[:2]

In [71]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans

In [72]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# property_clean_1[:2]

In [73]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# property_clean_1[:2]
# each_property_list_prep = [list(set(x)) for x in property_clean_1]
# each_property_list_prep[:2]

In [74]:
# dataframe_pred_prop = doc2vec(each_property_list_prep, 10)
# dataframe_pred_prop.head(3)

In [75]:
# dataframe_pred_prop.columns = ["pred_prop1","pred_prop2","pred_prop3","pred_prop4","pred_prop5",
#                               "pred_prop6","pred_prop7","pred_prop8","pred_prop9","pred_prop10",]
# data_3 = pd.concat([data_2, dataframe_pred_prop],axis=1)
# data_3.head(3)

In [76]:
# data_3.shape

In [77]:
# data_3.to_csv("data_nlp.csv")

-----

# context_timestamp特征处理

In [78]:
# data_3 = pd.read_csv("data_nlp.csv")

In [91]:
def time_value(time_stamp):
    hour = int(time_stamp.strftime("%H:%M:%S")[:2]) * 3600
    minus = int(time_stamp.strftime("%H:%M:%S")[3:5]) * 60
    sec = int(time_stamp.strftime("%H:%M:%S")[6:8])
    return hour + minus + sec

In [92]:
import datetime 
time_value_list = []
day_list = []
for i in range(len(data_1['context_timestamp'])):
    time_stamp = datetime.datetime.utcfromtimestamp(data_1['context_timestamp'][i])
    day = time_stamp.strftime("%Y-%m-%d")[-2:]
    time_value_list.append(time_value(time_stamp))
    day_list.append(day)

In [93]:
data_1['context_timestamp'] = time_value_list
#data_1["day"] = day_list
data_1.head(3)

,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [94]:
data_2 = data_1.copy(deep = True)

In [95]:
#增加一列 timestamp bin
#data_2['context_timestamp_bin'] = pd.cut(list(data_2['context_timestamp']),24)

In [96]:
data_2.columns

Index(['item_id', 'item_category_list', 'item_property_list', 'item_brand_id',
       'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'user_id_pred_0', 'item_brand_id_pred_0',
       'item_property_list_pred_0', 'predict_category_property_pred_0',
       'shop_id_pred_0', 'item_id_pred_0', 'day', 'hour', 'user_query_day',
       'user_query_day_hour', 'prop1', 'prop2', 'prop3', 'prop4', 'prop5',
       'prop6', 'prop7', 'prop8', 'prop9', 'prop10', 'pred_cate1',
       'pred_cate2', 'pred_cate3', 'pred_cate4', 'pred_cate5', 'pred_prop1',
       'pred_prop2', 'pred_prop3', 'pred_prop4', 'pred

In [97]:
data_2.shape

(496482, 59)

In [98]:
#进一步去掉无用的feature
data_3 = data_2.drop(['item_id','user_id','shop_id','day','item_property_list',
                     'item_brand_id','predict_category_property',],axis = 1)

In [99]:
level_features = ["user_star_level","shop_star_level",'context_page_id']

In [100]:
data_3[level_features].head(3)

,user_star_level,shop_star_level,context_page_id
0,3003,5002,4006
1,3006,5002,4001
2,3004,5002,4001


In [101]:
sales_price_ratio = []
for i in range(len(data_3)):
    sales_price_ratio.append(data_3.loc[i,"item_sales_level"] / data_3.loc[i,"item_price_level"])
sales_price_ratio[:2]    

i:\anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


[1.0, 1.0]

In [102]:
data_3["sales_price_ratio"] = sales_price_ratio

In [103]:
continu_features = ["item_price_level","item_sales_level","item_collected_level",
                 "shop_review_num_level","item_pv_level",'shop_score_description','shop_score_delivery',
               'shop_score_service','shop_review_positive_rate']

In [104]:
data_3[continu_features].head(3)

,item_price_level,item_sales_level,item_collected_level,shop_review_num_level,item_pv_level,shop_score_description,shop_score_delivery,shop_score_service,shop_review_positive_rate
0,3,3,4,4,14,1.0,1.0,1.0,1.0
1,3,3,4,4,14,1.0,1.0,1.0,1.0
2,3,3,4,4,14,1.0,1.0,1.0,1.0


In [105]:
data_3.columns

Index(['item_category_list', 'item_city_id', 'item_price_level',
       'item_sales_level', 'item_collected_level', 'item_pv_level',
       'user_gender_id', 'user_age_level', 'user_occupation_id',
       'user_star_level', 'context_timestamp', 'context_page_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'user_id_pred_0', 'item_brand_id_pred_0', 'item_property_list_pred_0',
       'predict_category_property_pred_0', 'shop_id_pred_0', 'item_id_pred_0',
       'hour', 'user_query_day', 'user_query_day_hour', 'prop1', 'prop2',
       'prop3', 'prop4', 'prop5', 'prop6', 'prop7', 'prop8', 'prop9', 'prop10',
       'pred_cate1', 'pred_cate2', 'pred_cate3', 'pred_cate4', 'pred_cate5',
       'pred_prop1', 'pred_prop2', 'pred_prop3', 'pred_prop4', 'pred_prop5',
       'pred_prop6', 'pred_prop7', 'pred_prop8', 'pred_prop9', 'pred_prop10',
       'sales_price_ratio'],
      

In [106]:
# feature_dummy = ["context_timestamp_bin","user_occupation_id","user_age_level",
#                 "user_gender_id",'item_city_id','item_category_list',"day",
#                  "item_pv_level","user_star_level","context_page_id","shop_review_num_level",
#                 "shop_star_level","item_sales_level","item_collected_level","item_price_level"]

feature_dummy = ["hour","user_occupation_id","user_age_level",
                "user_gender_id",'item_city_id','item_category_list',"user_star_level",
                 "shop_star_level","context_page_id"
                ]

In [107]:
# data_3[feature_dummy] = data_3[feature_dummy].astype(str)

In [108]:
data_3["user_occupation_id"] = data_3["user_occupation_id"].astype(str)
data_3["user_age_level"] = data_3["user_age_level"].astype(str)
data_3["user_gender_id"] = data_3["user_gender_id"].astype(str)
data_3['item_city_id'] = data_3['item_city_id'].astype(str)
data_3['item_category_list'] = data_3['item_category_list'].astype(str)
data_3['hour'] = data_3['hour'].astype(str)
data_3["user_star_level"] = data_3["user_star_level"].astype(str)
data_3["shop_star_level"] = data_3["shop_star_level"].astype(str)
data_3["context_page_id"] = data_3["context_page_id"].astype(str)

# data_3["item_pv_level"] = data_3["item_pv_level"].astype(str)
# data_3["user_star_level"] = data_3["user_star_level"].astype(str)
# data_3["context_page_id"] = data_3["context_page_id"].astype(str)
# data_3["shop_review_num_level"] = data_3["shop_review_num_level"].astype(str)
# data_3["shop_star_level"] = data_3["shop_star_level"].astype(str)
# data_3["item_sales_level"] = data_3["item_sales_level"].astype(str)
# data_3["item_collected_level"] = data_3["item_collected_level"].astype(str)
# data_3["item_price_level"] = data_3["item_price_level"].astype(str)

In [109]:
data_4 = pd.concat([data_3,pd.get_dummies(data_3[feature_dummy])],axis =1)
data_4 = data_4.drop(feature_dummy,axis = 1)

In [110]:
data_4.shape

(496482, 281)

In [111]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler() 
std.fit(data_4[continu_features].values)
data_4[continu_features] = std.transform(list(data_4[continu_features].values))       
print(data_4.shape)

(496482, 281)


In [112]:
data_4.head(3)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,context_timestamp,shop_review_num_level,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,...,context_page_id_4011,context_page_id_4012,context_page_id_4013,context_page_id_4014,context_page_id_4015,context_page_id_4016,context_page_id_4017,context_page_id_4018,context_page_id_4019,context_page_id_4020
0,-3.475377,-3.1379,-3.28112,-1.444348,7744,-4.162958,0.435111,1.222849,1.251912,1.012364,...,0,0,0,0,0,0,0,0,0,0
1,-3.475377,-3.1379,-3.28112,-1.444348,14432,-4.162958,0.435111,1.222849,1.251912,1.012364,...,0,0,0,0,0,0,0,0,0,0
2,-3.475377,-3.1379,-3.28112,-1.444348,68652,-4.162958,0.435111,1.222849,1.251912,1.012364,...,0,0,0,0,0,0,0,0,0,0


In [113]:
data_5 = data_4

In [119]:
#data_4 = data_4.drop(w2v_features,axis = 1)

In [120]:
#std_series = data_4.std(axis = 0)

In [121]:
# import math
# low_var_col = []
# for i in data_4.columns:
#     if std_series[i] < math.sqrt(.8 * (1 - .8)):
#         low_var_col.append(i)

In [122]:
#data_5 = data_4.drop(low_var_col,axis = 1)

In [123]:
#data_4 = data_4.drop("time",axis = 1)

In [114]:
data_5.columns

Index(['item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'context_timestamp', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description',
       ...
       'context_page_id_4011', 'context_page_id_4012', 'context_page_id_4013',
       'context_page_id_4014', 'context_page_id_4015', 'context_page_id_4016',
       'context_page_id_4017', 'context_page_id_4018', 'context_page_id_4019',
       'context_page_id_4020'],
      dtype='object', length=281)

In [115]:
def test_features(TYPE, data = data_5):
    if TYPE == 'all':
        data_6 = data_5
    elif TYPE == 'NOw2v':
        data_6 = data_5.drop(w2v_features,axis = 1)
    return data_6

In [116]:
data_6 = test_features('all')

------

In [117]:
train = data_6.loc[:478110,:]
test = data_6.loc[478111:,:]

-----

In [ ]:
train

In [ ]:
for x in train.columns:
    if train[x].dtype != 'float64' :
        print('Survival Correlation by:', x)
        print(data1[[x, Target[0]]].groupby(x, as_index=False).mean())
        print('-'*10, '\n')
print(pd.crosstab(data1['Title'],data1[Target[0]]))

In [118]:
train.shape

(478111, 281)

In [119]:
test.shape

(18371, 281)

In [120]:
train_y.shape

(478111,)

In [121]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, \
                    ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [122]:
MLA = [
#     ensemble.AdaBoostClassifier(),
#     ensemble.BaggingClassifier(),
#     ensemble.ExtraTreesClassifier(),
#     ensemble.GradientBoostingClassifier(),
#     ensemble.RandomForestClassifier(),
#     gaussian_process.GaussianProcessClassifier(),
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(),
#     linear_model.Perceptron(),
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
    LGBMClassifier(num_leaves=63, max_depth=7, n_estimators=80, n_jobs=20)
    #LGBMClassifier()
    ]

In [123]:
MLA

[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
         learning_rate=0.1, max_depth=7, min_child_samples=20,
         min_child_weight=0.001, min_split_gain=0.0, n_estimators=80,
         n_jobs=20, num_leaves=63, objective=None, random_state=None,
         reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
         subsample_for_bin=200000, subsample_freq=1)]

In [124]:
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [125]:
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, 
                                        random_state = 42 ) 

MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train log_loss Mean', 
               'MLA Test log_loss Mean' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
for alg in MLA:
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    cv_results = model_selection.cross_validate(alg, train, 
                                                train_y,cv  = cv_split,
                                                scoring='neg_log_loss')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train log_loss Mean'] = -cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test log_loss Mean'] = -cv_results['test_score'].mean()   
    MLA_compare.loc[row_index, 'MLA Test log_loss 3*STD'] = -cv_results['test_score'].std()*3   #let's know the worst that can happen!
    
    alg.fit(train, train_y)
    MLA_predict = alg.predict(test)
    print(MLA_name,"complete...")
    
    row_index+=1

MLA_compare.sort_values(by = ['MLA Test log_loss Mean'], ascending = False, inplace = True)
MLA_compare

i:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
i:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


LGBMClassifier complete...


,MLA Name,MLA Parameters,MLA Train log_loss Mean,MLA Test log_loss Mean,MLA Time,MLA Test log_loss 3*STD
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.0771641,0.0892676,18.9735,-0.002482


In [126]:
predict_prob = alg.predict_proba(test)[:,1]

In [127]:
feature_importance = pd.concat([pd.DataFrame(data_6.columns),pd.DataFrame(alg.feature_importances_)],axis =1 )

In [128]:
feature_importance.columns = ["feature", "importance"]

In [130]:
feature_importance.sort_values(by="importance",ascending = False)

,feature,importance
4,context_timestamp,275
43,sales_price_ratio,266
8,shop_score_delivery,188
16,user_query_day,131
26,prop9,122
17,user_query_day_hour,116
38,pred_prop6,107
30,pred_cate3,106
12,item_property_list_pred_0,105
9,shop_score_description,101


In [131]:
submission = pd.read_table("round1_ijcai_18_result_demo_20180301.txt")

In [132]:
submission_final = pd.DataFrame(columns=submission.columns)

In [133]:
submission_final["instance_id"] = test_1["instance_id"]
submission_final["predicted_score"] = predict_prob
submission_final = submission_final.drop("instance_id predicted_score",axis = 1)

In [134]:
from datetime import date

In [135]:
today = date.today().strftime('%Y%m%d')

In [136]:
submission_final.to_csv("submission_"+today+".csv",index = False,sep = ' ')

In [137]:
submission_final

,instance_id,predicted_score
478111,2475218615076601065,0.007395
478112,398316874173557226,0.024130
478113,6586402638209028583,0.023045
478114,1040996105851528465,0.019526
478115,6316278569655873454,0.030586
478116,868158305045921978,0.023328
478117,5713520501786699854,0.026567
478118,932945015407923184,0.013929
478119,1919197847086752313,0.012108
478120,304887065966615346,0.011664
